In [16]:
import numpy as np
from scipy import sparse
from sklearn.metrics import mean_squared_error

# randomly choose some users and remove their rating as test data
def train_test_split(data, size=10):
    data = data.tolil()
    test = sparse.lil_matrix(np.zeros(data.shape))
    train = data.copy()
    for user in range(data.shape[0]):
        row = data[user, :].nonzero()[0]
        if len(row) == 0:
            continue
        test_ratings = np.random.choice(row, 
                                        size, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = data[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(len((train.multiply(test)).nonzero()[0]) == 0)
    return train, test

def predict_similarity(user_movie, similarity):
    norm = np.array(np.abs(train_user_sim).sum(axis=1))
    norm[0] = 1 # hack: index-0 is meaning less, just to avoid divide by zero
    user_movie = sparse.csc_matrix(user_movie)
    similarity = sparse.csc_matrix(similarity)
    return similarity * user_movie / norm

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)